<h1>Soal 1: Pemahaman</h1>

1. Sebutkan apa saja yang termasuk preprocessing data!
2. Jelaskan beberapa cara imputing missing value!
3. Kapan kita perlu melakukan feature centering dan scaling?
4. Bagaimana Data Science Workflow?

Jawab:

1. - Binarization -> mengubah variabel numerik ke dalam nilai boolean (0 dan 1).
  - Mean Removal -> menghilangkan rata-rata, jadi variabel berada di tengah-tengah pada angka 0.
  - Scaling -> scale beberapa variabel yang memiliki nilai yang sangat bervariasi dan random dalam dataset yang masih mentah.
  - Normalization -> memodifikasi nilai dalam variabel sehingga kita dapat mengukurnya dalam skala umum.
  - Label encoding -> mengonversi label kata menjadi angka.
  

2. - Menggunakan kelas SimpleImputer().
  - Ketika membuat objek dari kelas SimpleImputer, terdapat beberapa parameter yang harus diatur:
    * missing_values : nilai data yang digunakan sebagai penanda bahwa nilai aslinya tidak ada (missing); dalam hal ini NaN (np.nan)
    * strategy : dalam hal ini rata-rata kolom ('mean'), bisa juga menggunakan 'median', 'most_frequent' (modus), atau 'constant'
  - Selanjutnya obyek imputer harus di fit berdasarkan kolom yang bersangkutan menggunakan metode fit(), dalam hal ini X kolom indeks 1 (age) dan 2 (wage). Nilai kosong pada kedua kolom itu akan diubah menggunakan metode transform().


3. Saat algoritma / Machine Learning distance-based atau terdapat perhitungan jarak antar data point agar influence tiap kolom dalam pembuatan model sama.


4. Workflow Data Science, pertama mengambil dari resources data, kemudian dilakukan processing bisa EDA, Data Analysis, Feature Engineering -> Modeling -> Deploy -> Monitor.

<h1>Soal 2: Pengaplikasian</h1>

Selamat, sampai tahap ini kalian telah belajar banyak tentang data science, dari mulai python, data manipulasi, visualisasi, dan pembuatan model. Sekarang saatnya untuk mengaplikasikan semuanya.

Download dan gunakan data [titanic.csv](https://drive.google.com/uc?export=download&id=15-XQkmqj2UlFQH7rASJH1oxWST6o9mbm) sebagai data untuk pembuatan model ML. Pahami betul data ini dengan melakukan EDA (Explolatory Data Analaysis), Visualisasi, Data Analysis, Preprocessing Data, dan Modeling.

<b>(Optional)</b> Download dan gunakan data [titanic_test.csv](https://drive.google.com/uc?export=download&id=15-XQkmqj2UlFQH7rASJH1oxWST6o9mbm) untuk mengetest model kalian dengan melakukan prediksi terhadap data tersebut. Submit hasil prediksinya ke kaggle dan lihat scorenya. https://www.kaggle.com/c/titanic/submit

![alt text](https://drive.google.com/uc?id=1SyflO2YeuCdYhxwPF6CAtMeaNN1qTaFx)

In [20]:
from google.colab import files
uploaded = files.upload()
import io

import pandas as pd
import numpy as np 
import math 
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

Saving titanic.csv to titanic (1).csv
Saving titanic_test.csv to titanic_test (1).csv


In [21]:
dp = pd.read_csv(io.BytesIO(uploaded['titanic.csv']))
test = pd.read_csv(io.BytesIO(uploaded['titanic_test.csv']))

Y = dp['Survived'].copy()
X = dp[dp.columns.difference(['Survived'])].copy()

In [22]:
X.describe()

,Age,Fare,Parch,PassengerId,Pclass,SibSp
count,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,29.699118,32.204208,0.381594,446.000000,2.308642,0.523008
std,14.526497,49.693429,0.806057,257.353842,0.836071,1.102743
min,0.420000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,20.125000,7.910400,0.000000,223.500000,2.000000,0.000000
50%,28.000000,14.454200,0.000000,446.000000,3.000000,0.000000
75%,38.000000,31.000000,0.000000,668.500000,3.000000,1.000000
max,80.000000,512.329200,6.000000,891.000000,3.000000,8.000000


In [23]:
combined_data_set = [X, test]

In [24]:
for data_set in combined_data_set:
    data_set['Sex'] = data_set['Sex'].replace({'male': 1,'female': 0})

In [25]:
for data_set in combined_data_set:
    data_set['S'] = np.where(data_set['Embarked'] == 'S', 1, 0)
    data_set['C'] = np.where(data_set['Embarked'] == 'C', 1, 0)
    data_set['Q'] = np.where(data_set['Embarked'] == 'Q', 1, 0)
    data_set.drop('Embarked', axis = 1, inplace = True)

In [26]:
for data_set in combined_data_set:
    data_set['Titles'] = data_set['Name'].str.extract(' ([A-Za-z]+)\.', expand = False)

In [27]:
titles_mapping = {'Mr': 0, 'Miss': 1, 'Mrs': 2}
for data_set in combined_data_set:
    data_set['Titles'] = data_set['Titles'].map(titles_mapping)
    data_set['Titles'] = data_set['Titles'].fillna(3)

## Fill the missing ages with median based on their title and normalize age.

In [28]:
for data_set in combined_data_set:
    data_set['Mr'] = np.where(data_set['Titles'] == 0, 1, 0)
    data_set['Miss'] = np.where(data_set['Titles'] == 1, 1, 0)
    data_set['Mrs'] = np.where(data_set['Titles'] == 2, 1, 0)
    data_set['Others'] = np.where(data_set['Titles'] == 3, 1, 0)
    data_set.drop('Titles', axis = 1, inplace = True)

In [29]:
IDs = test['PassengerId']

for data_set in combined_data_set:
      data_Parch =(data_set['Parch']-data_set['Parch'].min())/(data_set['Parch'].max() - data_set['Parch'].min())
      data_set['Parch'] = data_Parch
      data_SibSp =(data_set['SibSp']-data_set['SibSp'].min())/(data_set['SibSp'].max() - data_set['SibSp'].min())
      data_set['SibSp'] = data_SibSp

In [30]:
for data_set in combined_data_set:
    data_set['Fare'].fillna(data_set.groupby('Pclass')['Fare'].transform('median'), inplace = True)

for data_set in combined_data_set:
    data_set.loc[data_set['Fare'] <= 17, 'Fare'] = 0    
    data_set.loc[data_set['Fare'] > 17 & (data_set['Fare'] <= 30), 'Fare'] = 1 
    data_set.loc[data_set['Fare'] > 30 & (data_set['Fare'] <= 100), 'Fare'] = 2
    data_set.loc[data_set['Fare'] > 100 ,'Fare'] = 3

    data_set['0_Fare'] = np.where(data_set['Fare'] == 0, 1, 0)

    data_set['1_Fare'] = np.where(data_set['Fare'] == 1, 1, 0)
    
    data_set['2_Fare'] = np.where(data_set['Fare'] == 2, 1, 0)

    data_set['3_Fare'] = np.where(data_set['Fare'] == 3, 1, 0)
    
    data_set.drop('Fare', axis = 1, inplace = True)

In [31]:
for data_set in combined_data_set:
    data_set['1_class'] = np.where(data_set['Pclass'] == 1, 1, 0)

    data_set['2_class'] = np.where(data_set['Pclass'] == 2, 1, 0)

    data_set['3_class'] = np.where(data_set['Pclass'] == 3, 1, 0)

    data_set.drop('Pclass', axis = 1, inplace = True)

In [32]:
for data_set in combined_data_set:
    data_set.drop(['Cabin', 'Ticket', 'Name', 'PassengerId', 'Age', 'SibSp', 'Parch', '1_Fare', '2_Fare', 'Q'], axis = 1, inplace = True)

In [33]:
model = RandomForestClassifier(n_estimators = 100)
model.fit(X,Y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
from sklearn.metrics import accuracy_score

accuracy_score(Y, model.predict(X))

0.8361391694725028

In [35]:
prediction = model.predict(test)
prediction = {'PassengerId': IDs, 'Survived': prediction}

pd.DataFrame(prediction).to_csv('prediction.csv', index = False)

In [36]:
X

,Sex,S,C,Mr,Miss,Mrs,Others,0_Fare,3_Fare,1_class,2_class,3_class
0,1,1,0,1,0,0,0,1,0,0,0,1
1,0,0,1,0,0,1,0,0,0,1,0,0
2,0,1,0,0,1,0,0,1,0,0,0,1
3,0,1,0,0,0,1,0,0,0,1,0,0
4,1,1,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,1,1,0,0,0,0,1,1,0,0,1,0
887,0,1,0,0,1,0,0,0,0,1,0,0
888,0,1,0,0,1,0,0,0,0,0,0,1
889,1,0,1,1,0,0,0,0,0,1,0,0
